# Web Scrapping Google parte 2

Este código utiliza selenium y BeautifulSoup para automatizar la extracción de información de páginas web, convirtiéndola en texto sin etiquetas HTML, y luego guarda los resultados en archivos de texto individuales.

selenium: Automatiza el navegador para interactuar con páginas web.

webdriver_manager: Administra automáticamente el controlador del navegador Chrome.

BeautifulSoup: Extrae y procesa el contenido HTML de las páginas.

sleep: Pausa la ejecución para permitir la carga completa de la página.

json: Permite leer el archivo enlaces_reforma_judicial.json y guardar datos en un formato estructurado.

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from time import sleep
import json

Configurar Selenium:
Utiliza un navegador Chrome en modo sin interfaz gráfica (--headless) para mayor eficiencia.


Acceder a la URL:
Abre la página web con driver.get(url).


Esperar carga de contenido:
Usa sleep(3) para dar tiempo a que se cargue la página.


Procesar HTML con BeautifulSoup:
Extrae el contenido de la página con get_text, eliminando etiquetas HTML.


Cerrar navegador:
Garantiza que el navegador se cierre con driver.quit().


Retornar resultados:
Devuelve un diccionario con la URL y el texto extraído, o un error si ocurre un problema.

In [ ]:
def extraer_info_pagina(url):
    try:
        chrome_options = Options()
        chrome_options.add_argument("--headless")  
        chrome_options.add_argument("--disable-gpu")
        
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        
        driver.get(url)
        
        sleep(3)
        
        html = driver.page_source
        
        soup = BeautifulSoup(html, 'html.parser')
        
        texto_sin_etiquetas = soup.get_text(separator="\n", strip=True)
        
        driver.quit()
        
        return {'url': url, 'texto_sin_etiquetas': texto_sin_etiquetas}
    
    except Exception as e:
        return {'url': url, 'error': str(e)}

Abrir archivo de texto:
Usa open en modo escritura ('w') y codificación UTF-8 para soportar caracteres especiales.


Escribir información:
Incluye la URL y el texto extraído sin etiquetas HTML.


Guardar y notificar:
Muestra un mensaje indicando el nombre del archivo guardado.


In [ ]:
def guardar_en_txt(informacion, nombre_archivo):
    with open(nombre_archivo, 'w', encoding='utf-8') as archivo:
        archivo.write(f"Información extraída de la URL: {informacion['url']}\n\n")
        archivo.write(f"Texto extraído sin etiquetas HTML:\n")
        archivo.write(f"{informacion['texto_sin_etiquetas']}\n\n")
        print(f"Información guardada en {nombre_archivo}")

Leer enlaces desde JSON:
Abre y carga el archivo enlaces_reforma_judicial.json. Si no existe, muestra un mensaje de error.

Validar enlaces:
Si no hay enlaces, finaliza la ejecución.

Procesar cada enlace:
Llama a extraer_info_pagina para cada enlace y guarda el resultado en un archivo de texto único.

Manejar errores:
Si ocurre un error al extraer información de una página, lo notifica en la consola.

In [ ]:
def main():
    try:
        with open('enlaces_reforma_judicial.json', 'r', encoding='utf-8') as f:
            links = json.load(f)
    except FileNotFoundError:
        print("No se encontraron enlaces guardados.")
        return

    if not links:
        print("No se encontraron enlaces.")
        return
    
    for idx, link in enumerate(links, 1):
        info_pagina = extraer_info_pagina(link)
        if 'error' in info_pagina:
            print(f"Error al extraer la información de {link}: {info_pagina['error']}")
        else:
            archivo_txt = f'informacion_extraida_{idx}.txt' 
            guardar_en_txt(info_pagina, archivo_txt)

if __name__ == '__main__':
    main()